In [26]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Load Data 
X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True) # human readable feature values

#TODO this needs to be chnaged --> Ordinal Encoding !
categorial = ['Country','Workclass','Marital Status', 'Occupation','Relationship', 'Race', 'Sex']
for a in categorial:
    enc= OrdinalEncoder()
    enc.fit(np.array(X_display[a].values).reshape(-1, 1))
    temp= enc.transform(np.array(X_display[a].values).reshape(-1, 1))
   
    X_display[a]= temp.reshape(-1)

In [34]:
from carla.data.catalog.online_catalog import OnlineCatalog
dataset = OnlineCatalog('adult')

In [35]:
dataset

In [36]:

from carla.models.catalog import MLModelCatalog


training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)
data_name = "adult"
dataset = OnlineCatalog(data_name)

balance on test set 0.23883245958934032, balance on test set 0.2408256880733945
Epoch 0/9
----------
train Loss: 0.3667 Acc: 0.8275

test Loss: 0.3555 Acc: 0.8236

Epoch 1/9
----------
train Loss: 0.3407 Acc: 0.8408

test Loss: 0.3460 Acc: 0.8428

Epoch 2/9
----------
train Loss: 0.3331 Acc: 0.8449

test Loss: 0.3323 Acc: 0.8427

Epoch 3/9
----------
train Loss: 0.3307 Acc: 0.8444

test Loss: 0.3286 Acc: 0.8486

Epoch 4/9
----------
train Loss: 0.3291 Acc: 0.8423

test Loss: 0.3239 Acc: 0.8483

Epoch 5/9
----------
train Loss: 0.3286 Acc: 0.8442

test Loss: 0.3244 Acc: 0.8484

Epoch 6/9
----------
train Loss: 0.3276 Acc: 0.8458

test Loss: 0.3322 Acc: 0.8458

Epoch 7/9
----------
train Loss: 0.3276 Acc: 0.8433

test Loss: 0.3229 Acc: 0.8495

Epoch 8/9
----------
train Loss: 0.3269 Acc: 0.8468

test Loss: 0.3382 Acc: 0.8480

Epoch 9/9
----------
train Loss: 0.3263 Acc: 0.8460

test Loss: 0.3306 Acc: 0.8427



In [29]:
# load catalog model
model_type = "ann"
ml_model = MLModelCatalog(
    dataset,
    model_type=model_type,
    load_online=True,
    backend="pytorch"
)


In [37]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

display(test_factual)

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.02174,0.0,...,0.0,1.0,1.0,1.0,0.0
1,0.452055,0.048052,0.800000,0.00000,0.0,...,0.0,1.0,0.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.00000,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.00000,0.0,...,0.0,0.0,1.0,0.0,1.0


In [38]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.342642,0.085492,0.841394,0.063218,0.041384,...,0.0,1.0,1.0,1.0,1.0
1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
2,0.373496,0.149461,0.618071,0.085554,0.085999,...,1.0,1.0,1.0,1.0,1.0
3,0.453341,0.188718,0.439467,0.039478,0.039485,...,1.0,0.0,0.0,1.0,1.0
4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [39]:
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
evaluation_measures = [
    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(benchmark.mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
    evaluation_catalog.AvgTime({"time": benchmark.timer}),
]

# now run all implemented measurements and create a
# DataFrame which consists of all results
results = benchmark.run_benchmark(evaluation_measures)


[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Exp